In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()
findspark.find()

'/content/spark-3.2.1-bin-hadoop3.2'

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [91]:
df = spark.read.csv("/content/drive/My Drive/house_full.csv")
df.printSchema()
df.show(20,truncate = 100)
df.count()
from pyspark.sql import functions as F
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)

+----------------------------------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
|                               _c0|                                                            _c1|                                         _c2|   _c3|    _c4|        _c5|
+----------------------------------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
|                             Title|                                                       Location|                                       Price|  Area|  Rooms|   Interior|
| Apartment Nieuwezijds Voorburgwal|            1012 RT Amsterdam (Burgwallen-Nieuwe Zijde) 

In [92]:
from pyspark.sql.functions import col
df=df.filter(col("_c0")!="Title")
df.show(5, truncate =100)

+---------------------------------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
|                              _c0|                                                            _c1|                                         _c2|   _c3|    _c4|        _c5|
+---------------------------------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
|Apartment Nieuwezijds Voorburgwal|            1012 RT Amsterdam (Burgwallen-Nieuwe Zijde)        |                €5,000 per month            |180 m²|4 rooms|  Furnished|
|    Apartment Frederiksplein 40 B|                  1017 XN Amsterdam (De Weteringschans)        |                €3,400 per month            |125 m²|3 rooms|Upholstered|
|    Apartment George Gershwinlaan|                             1082 MT Amsterdam (Zuidas)        |                €3,000 per month         

In [93]:
colomns = ['Title','Location','Price','Area','Rooms','Description']
df = df.toDF(*colomns)
df.show(20, truncate = 100)

+----------------------------------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
|                             Title|                                                       Location|                                       Price|  Area|  Rooms|Description|
+----------------------------------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
| Apartment Nieuwezijds Voorburgwal|            1012 RT Amsterdam (Burgwallen-Nieuwe Zijde)        |                €5,000 per month            |180 m²|4 rooms|  Furnished|
|     Apartment Frederiksplein 40 B|                  1017 XN Amsterdam (De Weteringschans)        |                €3,400 per month            |125 m²|3 rooms|Upholstered|
|     Apartment George Gershwinlaan|                             1082 MT Amsterdam (Zuidas)        |                €3,000 per month   

In [94]:
from pyspark.sql.functions import when,regexp_replace
df = df.withColumn('Area',regexp_replace("Area",'m²','')
    )
df=df.withColumn("Area",df.Area.cast("int"))
df=df.withColumnRenamed('Area','Area(m²)')
df.show(20,truncate=100)
df.printSchema()


+----------------------------------+---------------------------------------------------------------+--------------------------------------------+--------+-------+-----------+
|                             Title|                                                       Location|                                       Price|Area(m²)|  Rooms|Description|
+----------------------------------+---------------------------------------------------------------+--------------------------------------------+--------+-------+-----------+
| Apartment Nieuwezijds Voorburgwal|            1012 RT Amsterdam (Burgwallen-Nieuwe Zijde)        |                €5,000 per month            |     180|4 rooms|  Furnished|
|     Apartment Frederiksplein 40 B|                  1017 XN Amsterdam (De Weteringschans)        |                €3,400 per month            |     125|3 rooms|Upholstered|
|     Apartment George Gershwinlaan|                             1082 MT Amsterdam (Zuidas)        |                €3,000 pe

In [95]:
df = df.withColumn('Price',regexp_replace("Price",'€',''))
df = df.withColumn('Price',regexp_replace("Price",',',''))
df = df.withColumn('Price',regexp_replace("Price",'per',''))
df = df.withColumn('Price',regexp_replace("Price",'month',''))
df = df.withColumn('Price',df.Price.cast("int"))
df = df.withColumnRenamed('Price','Price(€)')
df.show(20,truncate=100)
df.printSchema()

+----------------------------------+---------------------------------------------------------------+--------+--------+-------+-----------+
|                             Title|                                                       Location|Price(€)|Area(m²)|  Rooms|Description|
+----------------------------------+---------------------------------------------------------------+--------+--------+-------+-----------+
| Apartment Nieuwezijds Voorburgwal|            1012 RT Amsterdam (Burgwallen-Nieuwe Zijde)        |    5000|     180|4 rooms|  Furnished|
|     Apartment Frederiksplein 40 B|                  1017 XN Amsterdam (De Weteringschans)        |    3400|     125|3 rooms|Upholstered|
|     Apartment George Gershwinlaan|                             1082 MT Amsterdam (Zuidas)        |    3000|     126|3 rooms|Upholstered|
|       Apartment Ruyschstraat 31 A|                       1091 BS Amsterdam (Weesperzijde)        |    3450|     150|4 rooms|  Furnished|
|        Apartment Bezaanja

In [96]:
df = df.withColumn('Rooms',regexp_replace("Rooms",'rooms',''))
df = df.withColumn('Rooms',regexp_replace("Rooms",'room',''))
df = df.withColumn('Rooms',df.Rooms.cast("int"))
df.show(20,truncate=100)
df.printSchema()

+----------------------------------+---------------------------------------------------------------+--------+--------+-----+-----------+
|                             Title|                                                       Location|Price(€)|Area(m²)|Rooms|Description|
+----------------------------------+---------------------------------------------------------------+--------+--------+-----+-----------+
| Apartment Nieuwezijds Voorburgwal|            1012 RT Amsterdam (Burgwallen-Nieuwe Zijde)        |    5000|     180|    4|  Furnished|
|     Apartment Frederiksplein 40 B|                  1017 XN Amsterdam (De Weteringschans)        |    3400|     125|    3|Upholstered|
|     Apartment George Gershwinlaan|                             1082 MT Amsterdam (Zuidas)        |    3000|     126|    3|Upholstered|
|       Apartment Ruyschstraat 31 A|                       1091 BS Amsterdam (Weesperzijde)        |    3450|     150|    4|  Furnished|
|        Apartment Bezaanjachtplein|     

In [97]:
from pyspark.sql import functions as F
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

+-----+--------+--------+--------+-----+-----------+
|Title|Location|Price(€)|Area(m²)|Rooms|Description|
+-----+--------+--------+--------+-----+-----------+
|    0|       0|       1|       0|    0|          0|
+-----+--------+--------+--------+-----+-----------+



The missing value of the price is because the webpage doesn't have the price

In [99]:
df.write.csv("/content/drive/My Drive/Labs/house_clean")